<a href="https://colab.research.google.com/github/TLG442/Research_Project/blob/electricity-consumption-prediction-model%2Fit21222740/Smart_Meter_Electricity_Consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
pythonafroz_electricity_smart_meter_data_from_india_path = kagglehub.dataset_download('pythonafroz/electricity-smart-meter-data-from-india')

print('Data source import complete.')


100%|██████████| 176M/176M [00:02<00:00, 80.6MB/s]


Extracting files...
Data source import complete.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")# This Python 3 environment comes with many helpful analytics libraries installed

In [4]:
pd.set_option("display.max_columns",None)

In [8]:
import os
import pandas as pd

# Assuming the dataset is stored in the path returned by kagglehub
dataset_path = pythonafroz_electricity_smart_meter_data_from_india_path

# List the files in the dataset directory
print(os.listdir(dataset_path))

# Load the CSV files
df_2020 = pd.read_csv(os.path.join(dataset_path, "CEEW - Smart meter data Bareilly 2020.csv"))
df_2021 = pd.read_csv(os.path.join(dataset_path, "CEEW - Smart meter data Bareilly 2021.csv"))

# Display the first few rows to ensure it's loaded correctly
print(df_2020.head())
print(df_2021.head())


['SM Cleaned Data MH2021.csv', 'SM Cleaned Data MH Aggregated.csv', 'CEEW - Smart meter data Mathura 2020.csv', 'CEEW - Smart meter data Bareilly 2020.csv', 'CEEW - Smart meter data Bareilly 2021.csv', 'SM Cleaned Data BR Aggregated.csv', 'SM Cleaned Data BR2019.csv', 'CEEW - Smart meter data Mathura 2019.csv']
           x_Timestamp  t_kWh  z_Avg Voltage (Volt)  z_Avg Current (Amp)  \
0  2020-01-01 00:00:00  0.002                251.26                 0.15   
1  2020-01-01 00:03:00  0.001                251.23                 0.15   
2  2020-01-01 00:06:00  0.001                251.55                 0.14   
3  2020-01-01 00:09:00  0.001                251.97                 0.14   
4  2020-01-01 00:12:00  0.002                252.03                 0.14   

   y_Freq (Hz) meter  
0        49.97  BR02  
1        49.94  BR02  
2        49.94  BR02  
3        50.09  BR02  
4        50.08  BR02  
           x_Timestamp  t_kWh  z_Avg Voltage (Volt)  z_Avg Current (Amp)  \
0  2021-01-02 00

In [9]:
from IPython.core.display import display, HTML
display(HTML('<h2 style="color:Blue"> <b> Smart Meter Data Bareilly 2020 </b> </h2> '))

display(df_2020.shape)
display(df_2020.head())

from IPython.core.display import display, HTML
display(HTML('<h2 style="color:Blue"> <b> Smart Meter Data Bareilly 2021 </b> </h2> '))

display(df_2021.shape)
df_2021.head()

(6627360, 6)

,x_Timestamp,t_kWh,z_Avg Voltage (Volt),z_Avg Current (Amp),y_Freq (Hz),meter
0,2020-01-01 00:00:00,0.002,251.26,0.15,49.97,BR02
1,2020-01-01 00:03:00,0.001,251.23,0.15,49.94,BR02
2,2020-01-01 00:06:00,0.001,251.55,0.14,49.94,BR02
3,2020-01-01 00:09:00,0.001,251.97,0.14,50.09,BR02
4,2020-01-01 00:12:00,0.002,252.03,0.14,50.08,BR02


(3948960, 6)

,x_Timestamp,t_kWh,z_Avg Voltage (Volt),z_Avg Current (Amp),y_Freq (Hz),meter
0,2021-01-02 00:00:00,0.002,253.36,0.25,50.09,BR02
1,2021-01-02 00:03:00,0.002,253.87,0.25,50.11,BR02
2,2021-01-02 00:06:00,0.020,253.25,1.67,50.14,BR02
3,2021-01-02 00:09:00,0.045,252.20,3.52,50.12,BR02
4,2021-01-02 00:12:00,0.044,252.28,3.53,50.07,BR02


In [29]:
import numpy as np
import pandas as pd

# Choose the appropriate DataFrame: df_2020 or df_2021
df = df_2020  # Or df = df_2021, depending on which data you want to use
df = df_2020.sample(frac=0.01, random_state=42)

# Assuming df is your DataFrame after parsing and preprocessing
# We focus on the `t_kWh` column for prediction
data = df[['t_kWh']].values

# Define the fraction of data to be used for training
train_fraction = 0.8  # This is 80% for training, adjust if needed

# Calculate the split index
split_index = int(len(data) * train_fraction)

# Split the data into training and testing sets
train_data, test_data = data[:split_index], data[split_index:]

# Print the shapes of train and test sets for verification
print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")


Training data shape: (53019, 1)
Testing data shape: (13255, 1)


In [30]:

# Scale the data (MinMax scaling)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

# Function to create sequences for the LSTM
def create_sequences(data, sequence_length):
    xs = []
    ys = []
    for i in range(len(data) - sequence_length):
        x = data[i:i+sequence_length]
        y = data[i+sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Set the sequence length (e.g., 60 timesteps)
sequence_length = 60

# Create training sequences
X_train, y_train = create_sequences(scaled_train_data, sequence_length)

# Create test sequences
X_test, y_test = create_sequences(scaled_test_data, sequence_length)

# Reshape for LSTM input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Check the shape
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (52959, 60, 1)
Test set shape: (13195, 60, 1)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Build the LSTM Model
model = Sequential()

# Add LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())  # Batch Normalization

model.add(LSTM(units=50, return_sequences=False))
model.add(BatchNormalization())

# Add Dropout layer to prevent overfitting
model.add(Dropout(0.2))

# Add Dense output layer
model.add(Dense(units=1))  # Predicting a single value (t_kWh)

# Step 2: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [32]:
# Step 3: Train the Model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=5,  # Adjust the number of epochs based on your needs
                    batch_size=32,
                    verbose=1)  # This enables full details of each epoch


Epoch 1/5
1655/1655 [==============================] - 73s 42ms/step - loss: 0.0277 - val_loss: 0.0256
Epoch 2/5
1655/1655 [==============================] - 73s 44ms/step - loss: 0.0100 - val_loss: 0.0212
Epoch 3/5
1655/1655 [==============================] - 74s 45ms/step - loss: 0.0099 - val_loss: 0.0145
Epoch 4/5
1655/1655 [==============================] - 71s 43ms/step - loss: 0.0099 - val_loss: 0.0100
Epoch 5/5
1655/1655 [==============================] - 74s 45ms/step - loss: 0.0099 - val_loss: 0.0100


In [35]:

# Step 4: Evaluate the Model
# Evaluate the performance on the test set
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

413/413 [==============================] - 6s 15ms/step - loss: 0.0100
Test Loss: 0.009992101229727268
